## Consolidate Financial and Macroeconomic Data

In [1]:
import pandas as pd
import os

# --- Configuration ---
DATA_FOLDER = "data"
FINANCIAL_FILES = {
    'sp500': os.path.join(DATA_FOLDER, 's&p_500_daily.csv'),
    'nasdaq': os.path.join(DATA_FOLDER, 'nasdaq_daily.csv'),
    'ftse100': os.path.join(DATA_FOLDER, 'ftse_100_daily.csv')
}
MACRO_FILES = {
    #'gdp': os.path.join(DATA_FOLDER, 'macro_gdp.csv'),
    'inflation': os.path.join(DATA_FOLDER, 'macro_inflation_cpi.csv'),
    'unemployment': os.path.join(DATA_FOLDER, 'macro_unemploymentrate.csv'),
    'vix': os.path.join(DATA_FOLDER, 'macro_vix.csv')
}

# --- Load and Combine Data ---

# Load the primary financial index (e.g., S&P 500)
# The CSV structure: Row 1=headers, Row 2=tickers, Row 3=Date placeholder, Row 4+=data
# We need to skip rows 1 and 2, then use column 0 (Price) as the date index
df_master = pd.read_csv(FINANCIAL_FILES['sp500'], skiprows=[1, 2], index_col=0, parse_dates=True)

# Rename the index to something more meaningful
df_master.index.name = 'Date'

# Select and rename key columns to avoid confusion
df_master = df_master[['Open', 'High', 'Low', 'Close', 'Volume']].add_prefix('sp500_')

# Load and merge other financial indices
for name, path in FINANCIAL_FILES.items():
    if name != 'sp500':
        df_temp = pd.read_csv(path, skiprows=[1, 2], index_col=0, parse_dates=True)
        df_temp.index.name = 'Date'
        df_master = df_master.merge(
            df_temp[['Close', 'Volume']].add_prefix(f'{name}_'),
            left_index=True,
            right_index=True,
            how='left'
        )

# Load and merge macroeconomic data
for name, path in MACRO_FILES.items():
    try:
        df_macro = pd.read_csv(path, index_col='DATE', parse_dates=True)
        # Rename the column to be specific
        df_macro.rename(columns={df_macro.columns[0]: f'macro_{name}'}, inplace=True)
        df_master = df_master.merge(df_macro, left_index=True, right_index=True, how='left')
    except FileNotFoundError:
        print(f"Warning: {path} not found, skipping {name} data")

# --- Clean and Preprocess ---

# Forward-fill the macroeconomic data to fill daily gaps
macro_cols = [col for col in df_master.columns if 'macro_' in col]
if macro_cols:
    df_master[macro_cols] = df_master[macro_cols].ffill()

# Use interpolation for any remaining gaps (e.g., in stock data on holidays)
df_master.interpolate(method='time', inplace=True)

# Drop any rows that still have missing values (typically at the very beginning)
df_master.dropna(inplace=True)

print("✅ Master DataFrame created successfully!")
print("Shape of the data:", df_master.shape)
print("\nColumn names:")
print(df_master.columns.tolist())
print("\nFirst 5 rows:")
print(df_master.head())
print("\nLast 5 rows:")
print(df_master.tail())
print("\nData types:")
print(df_master.dtypes)
print("\nDate range:")
print(f"From: {df_master.index.min()} To: {df_master.index.max()}")

✅ Master DataFrame created successfully!
Shape of the data: (2577, 12)

Column names:
['sp500_Open', 'sp500_High', 'sp500_Low', 'sp500_Close', 'sp500_Volume', 'nasdaq_Close', 'nasdaq_Volume', 'ftse100_Close', 'ftse100_Volume', 'macro_inflation', 'macro_unemployment', 'macro_vix']

First 5 rows:
             sp500_Open   sp500_High    sp500_Low  sp500_Close  sp500_Volume  \
Date                                                                           
2015-04-01  2067.629883  2067.629883  2048.379883  2059.689941    3543270000   
2015-04-02  2060.030029  2072.169922  2057.320068  2066.959961    3095960000   
2015-04-06  2064.870117  2086.989990  2056.520020  2080.620117    3302970000   
2015-04-07  2080.790039  2089.810059  2076.100098  2076.330078    3065510000   
2015-04-08  2076.939941  2086.689941  2073.300049  2081.899902    3265330000   

            nasdaq_Close  nasdaq_Volume  ftse100_Close  ftse100_Volume  \
Date                                                                 

## Final Dataset Summary

In [3]:
# Final verification and summary of the consolidated dataset
print("="*60)
print("FINAL DATASET SUMMARY")
print("="*60)

print(f"📊 Dataset Shape: {df_master.shape[0]:,} rows × {df_master.shape[1]} columns")
print(f"📅 Date Range: {df_master.index.min().strftime('%Y-%m-%d')} to {df_master.index.max().strftime('%Y-%m-%d')}")
print(f"⏱️  Duration: {(df_master.index.max() - df_master.index.min()).days:,} days")

print("\n📈 Data Categories:")
financial_cols = [col for col in df_master.columns if any(x in col for x in ['sp500', 'nasdaq', 'ftse'])]
macro_cols = [col for col in df_master.columns if 'macro_' in col]
# sentiment_cols = [col for col in df_master.columns if 'gdelt' in col]

print(f"  • Financial Data: {len(financial_cols)} columns")
print(f"  • Macroeconomic Data: {len(macro_cols)} columns") 
#print(f"  • Sentiment Data: {len(sentiment_cols)} columns")

print("\n🔍 Data Quality Check:")
print(f"  • Total missing values: {df_master.isnull().sum().sum()}")
print(f"  • Rows with any missing values: {df_master.isnull().any(axis=1).sum()}")

print("\n📋 Column Summary:")
for i, col in enumerate(df_master.columns, 1):
    missing = df_master[col].isnull().sum()
    print(f"  {i:2}. {col:<20} - Missing: {missing:4} ({missing/len(df_master)*100:.1f}%)")

print("\n✅ Dataset is ready for modeling and analysis!")
print("="*60)

FINAL DATASET SUMMARY
📊 Dataset Shape: 2,577 rows × 12 columns
📅 Date Range: 2015-04-01 to 2025-06-30
⏱️  Duration: 3,743 days

📈 Data Categories:
  • Financial Data: 9 columns
  • Macroeconomic Data: 3 columns

🔍 Data Quality Check:
  • Total missing values: 0
  • Rows with any missing values: 0

📋 Column Summary:
   1. sp500_Open           - Missing:    0 (0.0%)
   2. sp500_High           - Missing:    0 (0.0%)
   3. sp500_Low            - Missing:    0 (0.0%)
   4. sp500_Close          - Missing:    0 (0.0%)
   5. sp500_Volume         - Missing:    0 (0.0%)
   6. nasdaq_Close         - Missing:    0 (0.0%)
   7. nasdaq_Volume        - Missing:    0 (0.0%)
   8. ftse100_Close        - Missing:    0 (0.0%)
   9. ftse100_Volume       - Missing:    0 (0.0%)
  10. macro_inflation      - Missing:    0 (0.0%)
  11. macro_unemployment   - Missing:    0 (0.0%)
  12. macro_vix            - Missing:    0 (0.0%)

✅ Dataset is ready for modeling and analysis!


## Create Time-Series & Crisis Features

In [5]:
import pandas as pd

# This assumes your master dataframe is named df_master

# --- Create Time-Series Features ---
# Moving Averages for the S&P 500 close price
df_master['sp500_ma_7'] = df_master['sp500_Close'].rolling(window=7).mean()
df_master['sp500_ma_30'] = df_master['sp500_Close'].rolling(window=30).mean()

# Lagged Values for the S&P 500 close price
for i in range(1, 4): # Create 3 lag features (t-1, t-2, t-3)
    df_master[f'sp500_lag_{i}'] = df_master['sp500_Close'].shift(i)

# --- Label Crisis Periods ---
# Define crisis periods (start_date, end_date)
# Using dates from your proposal [cite: 351, 402]
crisis_periods = {
    '2015_China_Market_Crash': ('2015-06-15', '2016-02-11'),
    '2018_Volatility': ('2018-10-01', '2018-12-31'),
    'COVID_Crash': ('2020-02-19', '2020-03-23'),
    '2022_Inflation_Crash': ('2022-01-01', '2022-12-31')
}

# Create the 'is_crisis' column, initialized to 0
df_master['is_crisis'] = 0

for crisis, (start, end) in crisis_periods.items():
    df_master.loc[start:end, 'is_crisis'] = 1

# Clean up by dropping initial rows with NaNs from lags/MA
df_master.dropna(inplace=True)

print("✅ Time-series and crisis features added!")
print("Number of crisis days labeled:", df_master['is_crisis'].sum())

# Show comprehensive date range information
print(f"\n📅 FULL DATASET DATE RANGE:")
print(f"   Start: {df_master.index.min().strftime('%Y-%m-%d')}")
print(f"   End: {df_master.index.max().strftime('%Y-%m-%d')}")
print(f"   Total days: {len(df_master):,}")

# Show first and last few rows to confirm full range
print(f"\n🔍 FIRST 3 ROWS (earliest dates):")
print(df_master[['sp500_Close', 'is_crisis']].head(3))

print(f"\n🔍 LAST 3 ROWS (latest dates):")
print(df_master[['sp500_Close', 'is_crisis']].tail(3))

# Show crisis period breakdown
print(f"\n📊 CRISIS PERIOD BREAKDOWN:")
for crisis, (start, end) in crisis_periods.items():
    crisis_data = df_master.loc[start:end] if start in df_master.index and end in df_master.index else pd.DataFrame()
    crisis_count = len(crisis_data)
    print(f"   {crisis}: {crisis_count} days ({start} to {end})")

print(f"\n📈 TOTAL CRISIS vs NON-CRISIS DAYS:")
crisis_counts = df_master['is_crisis'].value_counts()
print(f"   Non-Crisis (0): {crisis_counts.get(0, 0):,} days")
print(f"   Crisis (1): {crisis_counts.get(1, 0):,} days")

✅ Time-series and crisis features added!
Number of crisis days labeled: 499

📅 FULL DATASET DATE RANGE:
   Start: 2015-06-24
   End: 2025-06-30
   Total days: 2,519

🔍 FIRST 3 ROWS (earliest dates):
            sp500_Close  is_crisis
Date                              
2015-06-24  2108.580078          1
2015-06-25  2102.310059          1
2015-06-26  2101.489990          1

🔍 LAST 3 ROWS (latest dates):
            sp500_Close  is_crisis
Date                              
2025-06-26  6141.020020          0
2025-06-27  6173.069824          0
2025-06-30  6204.950195          0

📊 CRISIS PERIOD BREAKDOWN:
   2015_China_Market_Crash: 0 days (2015-06-15 to 2016-02-11)
   2018_Volatility: 63 days (2018-10-01 to 2018-12-31)
   COVID_Crash: 24 days (2020-02-19 to 2020-03-23)
   2022_Inflation_Crash: 0 days (2022-01-01 to 2022-12-31)

📈 TOTAL CRISIS vs NON-CRISIS DAYS:
   Non-Crisis (0): 2,020 days
   Crisis (1): 499 days


## Data Normalization & Final Prep


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# This assumes your fully-featured dataframe is named df_master

# Separate the target variable and the crisis flag
is_crisis_col = df_master['is_crisis']
# We scale features, not the crisis flag itself
features_to_scale = df_master.drop(columns=['is_crisis'])

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the features
scaled_features = scaler.fit_transform(features_to_scale)

# Create a new DataFrame with the scaled features
df_scaled = pd.DataFrame(scaled_features, index=df_master.index, columns=features_to_scale.columns)

# Add the 'is_crisis' column back to the scaled DataFrame
df_scaled['is_crisis'] = is_crisis_col

print("✅ Data successfully scaled!")
print("Shape of scaled data:", df_scaled.shape)
print("\nSample of scaled data:")
print(df_scaled.head())

df_scaled.to_csv('data/final_model_ready_data_only_timeSeries.csv')


✅ Data successfully scaled!
Shape of scaled data: (2519, 18)

Sample of scaled data:
            sp500_Open  sp500_High  sp500_Low  sp500_Close  sp500_Volume  \
Date                                                                       
2015-06-24    0.066572    0.063666   0.068382     0.063873      0.310978   
2015-06-25    0.063432    0.061592   0.066824     0.062440      0.322218   
2015-06-26    0.061748    0.059962   0.065358     0.062253      0.503730   
2015-06-29    0.060833    0.057607   0.056483     0.052232      0.368762   
2015-06-30    0.052246    0.052032   0.056409     0.053482      0.408814   

            nasdaq_Close  nasdaq_Volume  ftse100_Close  ftse100_Volume  \
Date                                                                     
2015-06-24      0.053131       0.009885       0.475687        0.177667   
2015-06-25      0.052497       0.009734       0.466178        0.174231   
2015-06-26      0.050529       0.033634       0.452274        0.158796   
2015-06-29  